### SQLAlchemy - ORM API

Based on the SQLAlchemy <a href="https://docs.sqlalchemy.org/en/14/tutorial/">documentation</a>.

Please, check the [ex1_sqlalchemy.ipynb](ex1_sqlalchemy.ipynb) for an general introduction to the use of SQLAlchemy.

---

**NOTE:** this code is intended to show hot to use the **ORM** API for creating and manipulating databases in SQLAlchemy. In [ex2_sqlalchemy_Core.ipynb](ex2_sqlalchemy_Core.ipynb), we discuss how to use the **Core** API for performing the same tasks. Most of the time, you can choose only one approach to work with databases in SQLAlchemy.

In [1]:
# Uncomment for installing the SQL Alchemy library.
#!pip install sqlalchemy

In [2]:
# importing the library
import sqlalchemy
# checking the version - just for demonstration
sqlalchemy.__version__ 

'1.4.22'

In [3]:
# importing necessary libraries
from sqlalchemy import create_engine

# Engine object refering an in-memory SQLite database
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

### Database metadata

The central element of both SQLAlchemy Core and ORM is the `SQL Expression Language` which allows for fluent, composable construction of SQL queries. The foundation for these queries are Python objects that represent database concepts like `tables` and `columns`. These objects are known collectively as **database metadata**.

The most common foundational objects for database metadata in SQLAlchemy are known as `MetaData`, `Table`, and `Column`. 

This notebook illustrate how these objects are used in an ORM-oriented style, following the reference documentation.

 ### ORM - Defining Table Metadata
 ---
We will reproduce the same example from the [Core notebook](ex2_sqlalchemy_Core.ipynb), but using an ORM-centric configuration paradigm.

When using the ORM, the process by which we declare `Table` metadata is usually combined with the process of declaring **mapped classes**. The `mapped class` is any Python class we would like to create, which will then have attributes on it that will be linked to the columns in a database table. While there are a few varieties of how this is achieved, the most common style is known as <a href="https://docs.sqlalchemy.org/en/14/orm/declarative_config.html">declarative</a>, and allows us to declare our `user-defined classes` and `Table` metadata at once.

### ORM - Setting up the Registry

The ORM API encapsulates the `MetaData` collection into an object known as the `registry`. We use `registry()` to create the object. Then, we can check its associated `metadata` object

In [4]:
# importing necessary libraries
from sqlalchemy.orm import registry
# creating the ORM registry
mapper_registry = registry()
# checking the metadata associated with the registry
mapper_registry.metadata

MetaData()

### Declaring Table objects and mapped classes

Instead of declaring `Table` objects directly, we can declare them indirectly through directives applied to mapped classes. Each mapped class descends from a common base class known as the **declarative base**. We get a new declarative base from the registry using the `registry.generate_base()` method:

In [5]:
# obtaining the declarative Base class for all other mapped classes
Base = mapper_registry.generate_base()

**IMPORTANT:** The steps of creating the registry and “declarative base” classes can be combined into one step using the historically familiar `declarative_base()` function:
```
from sqlalchemy.orm import declarative_base
Base = declarative_base()
```

### ORM - Declaring Mapped Classes
---
The `Base` object is a Python class which will serve as the base class for the ORM mapped classes we declare. We can now define ORM mapped classes for the user and address table in terms of new classes `User` and `Address`:



In [6]:
# importing necessary libraries
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship

# class User (inherits from Base)
class User(Base):
  # the table name to associate this class to
  __tablename__ = 'user_account'
  # metadata
  id = Column(Integer, primary_key=True)
  name = Column(String(30))
  fullname = Column(String)

  # relationship mapping ONE user to MANY addressess
  # this is optional at this point => but addressed later in this code
  addresses = relationship("Address", back_populates="user")

  # this method returns a descriptive string of the current class' instance
  def __repr__(self):
    return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

# class Address (inherits from Base)
class Address(Base):
  # the table name to associate this class to
  __tablename__ = 'address'
  # metadata
  id = Column(Integer, primary_key=True)
  email_address = Column(String, nullable=False)
  user_id = Column(Integer, ForeignKey('user_account.id'))

  # relationship mapping MANY addressess to ONE user
  # this is optional at this point => but addressed later in this code
  user = relationship("User", back_populates="addresses")

  # this method returns a descriptive string of the current class' instance
  def __repr__(self):
    return f"Address(id={self.id!r}, email_address={self.email_address!r})"

The above two classes are now our mapped classes, and are available for use in ORM persistence and query operations. Mapped classes also include `Table` objects that were generated as part of the declarative mapping process, and are equivalent to the ones that we declared directly in the previous Core section. We can see these `Table` objects from a declarative mapped class using the `.__table__` attribute:

In [7]:
# checking the Table object associated with a mapped class
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

In [8]:
# checking the Table object associated with a mapped class
Address.__table__

Table('address', MetaData(), Column('id', Integer(), table=<address>, primary_key=True, nullable=False), Column('email_address', String(), table=<address>, nullable=False), Column('user_id', Integer(), ForeignKey('user_account.id'), table=<address>), schema=None)

### ORM - Emitting DDL to the database
---
Emitting DDL with our ORM mapped classes is not different from using the Core approach. If we wanted to emit DDL for the `Table` objects we’ve created as part of our declaratively mapped classes, we still can use `MetaData.create_all()` as before.

In our case, we have already generated the `user` and `address` tables in our SQLite database. If we had not done so already, we would be free to make use of the MetaData associated with our registry and ORM declarative base class in order to do so, using `MetaData.create_all()`:


In [9]:
# emit CREATE statements given ORM registry
mapper_registry.metadata.create_all(engine)
# OR
# using the MetaData object present on the declarative base
# Base.metadata.create_all(engine)

2021-08-17 16:06:35,486 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-17 16:06:35,492 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2021-08-17 16:06:35,494 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-17 16:06:35,496 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2021-08-17 16:06:35,499 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-17 16:06:35,501 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2021-08-17 16:06:35,502 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-17 16:06:35,503 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2021-08-17 16:06:35,505 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-17 16:06:35,510 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2021-08-17 16:06:35,523 INFO sqlalchemy.engine.Engine [no key 0.01362s] ()
2021-08-17 16:06:35,528 INFO sqlalchemy.engine.Engine 
C

### ORM - Inserting rows
---
When using the ORM, the `Session` object is responsible for constructing Insert constructs and emitting them in a transaction. The way we instruct the `Session` to do so is by **adding object entries** to it; the `Session` then makes sure these new entries will be emitted to the database when they are needed, using a process known as a **flush**.

**Instances of Classes represent Rows**

With the ORM, we make direct use of the custom Python classes we defined previously. At the class level, the `User` and `Address` classes served as a place to define what the corresponding database tables should look like. These classes also serve as extensible data objects that we use to create and manipulate rows within a transaction as well. Below we will create three `User` objects each representing a potential database row to be inserted:

Classes have an automatically generated `__init__()` method that allows for parameterized construction of the objects (see <a href="https://docs.sqlalchemy.org/en/14/orm/mapping_styles.html#mapped-class-default-constructor">default constructor</a>). We are free to provide our own `__init__()` method as well.

In [10]:
# creating instances of User class through the implicit init() method
sandy = User(name="Sandy", fullname="Sandy Cheeks")
squidward = User(name="Squidward", fullname="Squidward Tentacles")
eugene = User(name="Eugene", fullname="Eugene H. Krabs")

In a similar manner as in our Core examples of Insert, we did not include a primary key (i.e. an entry for the id column), since we would like to make use of the auto-incrementing primary key feature of the database, SQLite in this case, which the ORM also integrates with. The value of the `id` attribute on the above objects, if we were to view it, displays itself as `None`:

In [11]:
# checking a given object through the repr() method
sandy

User(id=None, name='Sandy', fullname='Sandy Cheeks')

In [12]:
# checking a given object through the repr() method
squidward

User(id=None, name='Squidward', fullname='Squidward Tentacles')

The `id=None` value is provided by SQLAlchemy to indicate that the attribute has no value as of yet. SQLAlchemy-mapped attributes always return a value in Python and don’t raise `AttributeError` if they’re missing, when dealing with a new object that has not had a value assigned.

At the moment, our three objects above are said to be in a state called **transient** - they are not associated with any database state and are yet to be associated with a `Session` object that can generate INSERT statements for them.

### ORM: Opening the Session and adding objects
---
We will create a `Session` without using a context manager (and hence we must make sure we close it later!):

In [13]:
# importing necessary libraries
from sqlalchemy.orm import Session

# creating (opening) the session
session = Session(engine)

The objects are then added to the `Session` using the `Session.add()` method. When this is called, the objects are in a state known as **pending** and have not been inserted yet:

In [14]:
# adding the objects to the session for further insertion
session.add(sandy)
session.add(squidward)
session.add(eugene)

When we have pending objects, we can see this state by looking at a collection on the `Session` called `Session.new`.

A collection called `IdentitySet` is essentially a Python set that hashes on object identity in all cases (i.e., using Python built-in `id()` function, rather than the Python `hash()` function).



In [15]:
# checking the collection of pending objects
session.new

IdentitySet([User(id=None, name='Sandy', fullname='Sandy Cheeks'), User(id=None, name='Squidward', fullname='Squidward Tentacles'), User(id=None, name='Eugene', fullname='Eugene H. Krabs')])

The `Session` makes use of a pattern known as [unit of work](https://docs.sqlalchemy.org/en/14/glossary.html#term-unit-of-work). This generally means it accumulates changes one at a time, but does not actually communicate them to the database until needed. This allows it to make better decisions about how SQL DML should be emitted in the transaction based on a given set of pending changes. When it does emit SQL to the database to push out the current set of changes, the process is known as a **flush**.

We can illustrate the flush process manually by calling the `Session.flush()` method:

In [16]:
# flushing all new objects into the database
session.flush()

2021-08-17 16:06:35,633 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-17 16:06:35,639 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-08-17 16:06:35,642 INFO sqlalchemy.engine.Engine [generated in 0.00288s] ('Sandy', 'Sandy Cheeks')
2021-08-17 16:06:35,645 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-08-17 16:06:35,647 INFO sqlalchemy.engine.Engine [cached since 0.008432s ago] ('Squidward', 'Squidward Tentacles')
2021-08-17 16:06:35,649 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-08-17 16:06:35,652 INFO sqlalchemy.engine.Engine [cached since 0.01295s ago] ('Eugene', 'Eugene H. Krabs')


We observe the `Session` was first called upon to emit SQL, so it created a new transaction and emitted the appropriate INSERT statements for the two objects. The transaction now remains open until we call any of the `Session.commit()`, `Session.rollback()`, or `Session.close()` methods.

While `Session.flush()` may be used to manually push out pending changes to the current transaction, it is usually unnecessary as the `Session` features a behavior known as **autoflush**, which we will illustrate later. It also flushes out changes whenever `Session.commit()` is called.

**Autogenerated primary key attributes**

Once the rows are inserted, the objects are in a state known as **persistent**, where they are associated with the `Session object` in which they were added or loaded.

Another effect of the INSERT that occurred was that the ORM has retrieved the new primary key identifiers for each new object; internally it normally uses the `CursorResult.inserted_primary_key` accessor. We can check the `id` attribute of the new objects:

In [17]:
squidward.id

2

**Identity Map** 

The **identity map** is an in-memory store that links all objects currently loaded in memory to their primary key identity. We can observe this by retrieving one of the above objects using the `Session.get()` method, which will return an entry from the identity map if locally present, otherwise emitting a SELECT:

In [18]:
# retrieving data from an object through its identity map
some_squidward = session.get(User, 2)
some_squidward

User(id=2, name='Squidward', fullname='Squidward Tentacles')

**IMPORTANT:** The identity map maintains a unique instance of a particular Python object per a particular database identity, within the scope of a particular `Session` object. We may observe that the `some_squidward` refers to the same object as that of `squidward` previously:

In [19]:
some_squidward is squidward

True

**Committing**

There’s much more to say about how the Session works which will be discussed further. For now we will commit the transaction so that we can build up knowledge on how to SELECT rows before examining more ORM behaviors and features:

In [20]:

session.commit()

2021-08-17 16:06:35,703 INFO sqlalchemy.engine.Engine COMMIT


### ORM: Selecting rows

When using the ORM, particularly with a `select()` construct that’s composed against ORM entities, we will want to execute it using the `Session.execute(
)` method on the Session; using this approach, we continue to get `Row` objects from the result, however these rows are now capable of including complete entities, such as instances of the `User` class, as individual elements within each row:

In [21]:
# importing necessary libraries
from sqlalchemy import select

# SELECT statement
stmt = select(User).where(User.name == 'Squidward')
# executing the SELECT statement and retrieving the rows
with Session(engine) as session:
  # iterating over the rows (like a cursor)
  for row in session.execute(stmt):
    print(row)

2021-08-17 16:06:35,720 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-17 16:06:35,727 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2021-08-17 16:06:35,730 INFO sqlalchemy.engine.Engine [generated in 0.00364s] ('Squidward',)
(User(id=2, name='Squidward', fullname='Squidward Tentacles'),)
2021-08-17 16:06:35,734 INFO sqlalchemy.engine.Engine ROLLBACK


**Selecting ORM entities and columns**

ORM entities, such our `User` class as well as the column-mapped attributes upon it such as `User.name`, also participate in the SQL Expression Language system representing tables and columns. Below illustrates an example of SELECTing from the `User` entity, which ultimately renders in the same way as if we had used user_table directly:

In [22]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


When executing a statement through the ORM `Session.execute()` method, we are selecting from a full entity (such as `User`). 
```
# SELECT statement
stmt = select(User).where(User.name == 'Squidward')
# executing the SELECT statement and retrieving the rows
with Session(engine) as session:
  # iterating over the rows (like a cursor)
  for row in session.execute(stmt):
    print(row)
```
In this case, the entity itself is returned as a single element within each row. That is, when we fetch rows from the above statement, as there is only the `User` entity in the list of things to fetch, we get back `Row` objects that have only one element, which contain instances of the `User` class:

In [23]:
row = session.execute(select(User)).first()
row  # or row[0]

2021-08-17 16:06:35,759 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-17 16:06:35,763 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2021-08-17 16:06:35,765 INFO sqlalchemy.engine.Engine [generated in 0.00256s] ()


(User(id=1, name='Sandy', fullname='Sandy Cheeks'),)

Alternatively, we can select individual columns of an ORM entity as distinct elements within result rows, by using the class-bound attributes; when these are passed to a construct such as `select()`, they are resolved into the Column or other SQL expression represented by each attribute:

In [24]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


In [25]:
row = session.execute(select(User.name, User.fullname)).first()
row

2021-08-17 16:06:35,794 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2021-08-17 16:06:35,796 INFO sqlalchemy.engine.Engine [generated in 0.00214s] ()


('Sandy', 'Sandy Cheeks')

The approaches can also be mixed, as below where we SELECT the name attribute of the `User` entity as the first element of the row, and combine it with full `Address` entities in the second element:

In [26]:
# example of a SELECT combining User and Address attributes.
# NOTE: this example will return an empty set for now, as we didn't insert values into the Address class yet.
session.execute(
    select(User.name, Address).
    where(User.id==Address.user_id).
    order_by(Address.id)
).all()

2021-08-17 16:06:35,817 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2021-08-17 16:06:35,821 INFO sqlalchemy.engine.Engine [generated in 0.00416s] ()


[]

### ORM: Updating objects
---
When using the ORM, there are two ways in which the `Update` construct is used. The primary way is that it is emitted automatically as part of the *unit of work* process used by the `Session`, where an UPDATE statement is emitted on a per-primary key basis corresponding to individual objects that have changes on them. 

Supposing we loaded the `User` object for the `username Sandy` into a transaction (also showing off the `Select.filter_by()` method as well as the `Result.scalar_one()` method):

In [27]:
# SELECT statement to retrieve the instance Sandy from the User class
sandy = session.execute(select(User).filter_by(name="Sandy")).scalar_one()

2021-08-17 16:06:35,839 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2021-08-17 16:06:35,842 INFO sqlalchemy.engine.Engine [cached since 0.1154s ago] ('Sandy',)


In [28]:
sandy

User(id=1, name='Sandy', fullname='Sandy Cheeks')

The `sandy` object is a **proxy** to the row in the table, which means any modification to this object will be later reflected in the table. But while this not occurs, the object appears in the `session.dirty` collection.


In [29]:
# Updating an object
sandy.fullname = "Sandy Squirrel"

In [30]:
# checking the state of the object - "dirty" means the modifications are not yet reflected in the table
sandy in session.dirty

True

When the `Session` next emits a flush, an UPDATE will be emitted that updates this value in the database. As mentioned previously, a flush occurs automatically before we emit any SELECT, using a behavior known as **autoflush**. We can query directly for the `User.fullname` column from this row and we will get our updated value back:

In [31]:
# SELECT statement using the User.id as key (1 for Sandy)
sandy_fullname = session.execute(
    select(User.fullname).where(User.id == 1)
).scalar_one()

2021-08-17 16:06:35,903 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.id = ?
2021-08-17 16:06:35,906 INFO sqlalchemy.engine.Engine [generated in 0.00355s] ('Sandy Squirrel', 1)
2021-08-17 16:06:35,913 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2021-08-17 16:06:35,916 INFO sqlalchemy.engine.Engine [generated in 0.00343s] (1,)


In [32]:
print(sandy_fullname)

Sandy Squirrel


In [33]:
# checking the state of the object - "dirty == FALSE" means all modifications were reflected in the database
sandy in session.dirty

False

### ORM-enabled UPDATE statements

There’s a second way to emit UPDATE statements in terms of the ORM, which is known as an **ORM enabled UPDATE statement**. This allows the use of a generic SQL UPDATE statement that can affect many rows at once. For example to emit an UPDATE that will change the `User.fullname` column based on a value in the `User.name` column:

In [34]:
# importing necessary libraries
from sqlalchemy import update

# UPDATE statement
session.execute(
    update(User).
    where(User.name == "Sandy").
    values(fullname="Sandy Squirrel Extraordinaire")
)

2021-08-17 16:06:35,974 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2021-08-17 16:06:35,978 INFO sqlalchemy.engine.Engine [generated in 0.00369s] ('Sandy Squirrel Extraordinaire', 'Sandy')


### ORM: Deleting Objects

To round out the basic persistence operations, an individual ORM object may be marked for deletion by using the `Session.delete()` method. Let’s load up `Eugene Krabs` from the database:

In [35]:
ehkrabs = session.get(User, 3) 

2021-08-17 16:06:36,002 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2021-08-17 16:06:36,006 INFO sqlalchemy.engine.Engine [generated in 0.00460s] (3,)


We can mark the object for deletion. As is the case with other operations, nothing actually happens yet until a flush proceeds:



In [36]:
session.delete(ehkrabs)

Current ORM behavior is that the object stays in the `Session` until the flush proceeds, which as mentioned before occurs if we emit a query:

In [37]:
session.execute(select(User).where(User.name == "Eugene")).first()

2021-08-17 16:06:36,046 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2021-08-17 16:06:36,050 INFO sqlalchemy.engine.Engine [generated in 0.00359s] (3,)
2021-08-17 16:06:36,057 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2021-08-17 16:06:36,062 INFO sqlalchemy.engine.Engine [generated in 0.00475s] (3,)
2021-08-17 16:06:36,067 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2021-08-17 16:06:36,070 INFO sqlalchemy.engine.Engine [cached since 0.3438s ago] ('Eugene',)


Above, the SELECT we asked to emit was preceded by a DELETE, which indicated the pending deletion for `ehkrabs` proceeded. 

We can confirm the object was deleted by checking the current `Session` objects.



In [38]:
ehkrabs in session

False

**IMPORTANT:** just like the UPDATEs we made to the `sandy` object, every change we’ve made here is local to an ongoing transaction, which won’t become permanent if we don’t commit it. 

**ORM-enabled DELETE Statements**

Like UPDATE operations, there is also an ORM-enabled version of DELETE which we can illustrate by using the `delete()` construct with `Session.execute()`. It also has a feature by which non expired objects that match the given deletion criteria will be automatically marked as *deleted* in the `Session`:


In [39]:
# importing necessary libraries
from sqlalchemy import delete

# refresh the target object for demonstration purposes only, not needed for the DELETE
squidward = session.get(User, 2)

# DELETE statement
session.execute(delete(User).where(User.name == "Squidward"))

2021-08-17 16:06:36,110 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2021-08-17 16:06:36,115 INFO sqlalchemy.engine.Engine [cached since 0.1131s ago] (2,)
2021-08-17 16:06:36,122 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.name = ?
2021-08-17 16:06:36,125 INFO sqlalchemy.engine.Engine [generated in 0.00323s] ('Squidward',)


In [40]:
squidward in session

False

### [OPTIONAL - BUT IMPORTANT] Rolling back a session
---
The `Session` has a `Session.rollback()` method that as expected emits a ROLLBACK on the SQL connection in progress. However, it also has an effect on the objects that are currently associated with the `Session`.

In the previous example, with the object `sandy`, we changed the `.fullname` to read "Sandy Squirrel". We want to roll back this change. Calling `Session.rollback()` will not only roll back the transaction but also **expire all objects** currently associated with this `Session`, which will have the effect that they will refresh themselves when next accessed using a process known as [lazy loading](https://docs.sqlalchemy.org/en/14/glossary.html#term-lazy-loading):


In [41]:
# Rolling back the session
# session.rollback()

To view the “expiration” process more closely, we may observe that object `sandy` has no state left within its Python `__dict__`, with the exception of a special SQLAlchemy internal state object:

In [42]:
# sandy.__dict__

This is the “expired” state; accessing the attribute again will autobegin a new transaction and refresh `sandy` with the current database row:

In [43]:
# sandy.fullname

We may now observe that the full database row was also populated into the `__dict__` of the `sandy` object:

In [44]:
# sandy.__dict__

For **deleted objects**, when we earlier noted that `ehkrabs` was no longer in the session, that object’s identity is also restored:

In [45]:
# ehkrabs in session

and of course the database data is present again as well:


In [46]:
# session.execute(select(User).where(User.name == 'Eugene')).scalar_one() is ehkrabs

### ORM: Closing a Session

Within the above sections we used a `Session` object outside of a Python context manager (that is, we didn’t use the `with` statement). That’s fine, however if we are doing things this way, it’s best that we explicitly close out the `Session` when we are done with it:

In [47]:
# explicitly closing the Session
session.close()

2021-08-17 16:06:36,248 INFO sqlalchemy.engine.Engine ROLLBACK


___
### ORM: Relationships among objects
---
In this section, we will discuss how ORM interacts with mapped classes that refer to other objects. At the beginning of this code (and also reproduced below), the mapped class examples made use of a construct called `relationship()`. This construct defines a linkage between two different mapped classes, or from a mapped class to itself, the latter of which is called a *self-referential relationship*.

To describe the basic idea of `relationship()`, first we’ll review the mapping in short form, omitting the Column mappings and other directives:
```
# class User (inherits from Base)
class User(Base):
  # the table name to associate this class to
  __tablename__ = 'user_account'
  # metadata

  # relationship mapping ONE user to MANY addressess
  # this is optional at this point => but addressed later in this code
  addresses = relationship("Address", back_populates="user")

# class Address (inherits from Base)
class Address(Base):
  # the table name to associate this class to
  __tablename__ = 'address'
  # metadata

  # relationship mapping MANY addressess to ONE user
  # this is optional at this point => but addressed later in this code
  user = relationship("User", back_populates="addresses")
```

Above, the `User` class has an attribute `User.addresses` and the `Address` class has an attribute `Address.user`. The `relationship()` construct will be used to inspect the table relationships between the Table objects that are mapped to the `User` and `Address` classes. As the Table object representing the address table has a `ForeignKeyConstraint` which refers to the `user_account` table, the `relationship()` can determine unambiguously that **there is a one to many relationship from `User.addresses` to `User`; one particular row in the `user_account` table may be referred towards by many rows in the `address` table**.

**All one-to-many relationships naturally correspond to a many to one relationship in the other direction**, in this case the one noted by `Address.user`. The `relationship.back_populates parameter`, configured on both `relationship()` objects referring to the other name, establishes that **each of these two `relationship()` constructs should be considered to be complimentary to each other**.


**Persisting and loading Relationships**

Let's see what `relationship()` does to instances of objects. If we make a new `User` object, we can note that there is a Python list when we access the `.addresses` element:


In [48]:
# creating (inserting) a new User object
u1 = User(name='Larry', fullname='Larry the Lobster')
# checking the address list associated to this object
u1.addresses

[]

This object is a SQLAlchemy-specific version of Python list which has the ability to track and respond to changes made to it. The collection also appeared automatically when we accessed the attribute, even though we never assigned it to the object. This is similar to the behavior noted at *ORM: Inserting rows* where it was observed that column-based attributes to which we don’t explicitly assign a value also display as `None` automatically, rather than raising an `AttributeError` as would be Python’s usual behavior.

As the `u1` object is still transient and the list that we got from `u1.addresses` has not been mutated (i.e. appended or extended), it’s not actually associated with the object yet, but as we make changes to it, it will become part of the state of the `User` object.

The collection is specific to the `Address` class which is the only type of Python object that may be persisted within it. Using the `list.append()` method we may add an `Address` object:

In [49]:
# inserting an Address object related to User u1
a1 = Address(email_address="larry.lobster@sea.com")
u1.addresses.append(a1)

In [50]:
# checking again the address list associated to this object
u1.addresses

[Address(id=None, email_address='larry.lobster@sea.com')]

**Bidirectional navigation**

As we associated the `Address` object with the `User.addresses` collection of the u1 instance, another behavior also occurred, which is that the `User.addresses` relationship synchronized itself with the `Address.user` relationship, such that we can navigate not only from the `User` object to the `Address` object, we can also navigate from the `Address` object back to the *parent* `User` object:

In [51]:
# checking the User (parent object) associated with an Address object
a1.user

User(id=None, name='Larry', fullname='Larry the Lobster')

This synchronization occurred as a result of the `relationship.back_populates` parameter between the two `relationship()` objects. This parameter names another `relationship()` for which complementary attribute assignment / list mutation should occur. It will work equally well in the other direction, which is that if we create another `Address` object and assign to its `Address.user` attribute, that `Address` becomes part of the `User.addresses` collection on that `User` object:


In [52]:
# inserting another Address object associated with User u1
a2 = Address(email_address="larrylob@aol.com", user=u1)
u1.addresses

[Address(id=None, email_address='larry.lobster@sea.com'),
 Address(id=None, email_address='larrylob@aol.com')]

We actually made use of the `user`` parameter as a keyword argument in the `Address` constructor, which is accepted just like any other mapped attribute that was declared on the `Address` class. It is equivalent to assignment of the `Address.user` attribute after the fact:


In [53]:
# equivalent effect as a2 = Address(user=u1)
a2.user = u1
a2.user

User(id=None, name='Larry', fullname='Larry the Lobster')

**Cascading objects into the Session**

We now have a `User` and two `Address` objects that are associated in a bidirectional structure in memory, but these objects are said to be in the **transient** state until they are associated with a `Session` object.

We make use of the `Session.add()` method to the lead `User` object, so we add it and its related `Address` objects to the same `Session`. This behaviour is known as **save-update cascade**: the `Session` received a `User` object, and followed along the `User.addresses` relationship to locate all related `Address` objects.



In [54]:
# inserting User u1 into the session
session.add(u1)
u1 in session

True

In [55]:
# checking whether the Address objects associatd to User u1 are in the session 
a1 in session # do the same for a2

True

**IMPORTANT:** The three objects are now in the **pending** state; this means they are ready to be the subject of an INSERT operation but this has not yet proceeded. All three objects have no primary key assigned yet, and in addition, the `a1` and `a2` objects have an attribute called `user_id` which refers to the Column that has a `ForeignKeyConstraint` referring to the `user_account.id` column; these are also `None` as the objects are not yet associated with a real database row:

In [56]:
# printing the User ID (primary key) => None for now, as the object has not yet been inserted in the database
print(u1.id)
# the same applies to the User ID attached to the Address object "a1"
print(a1.user_id)

None
None


**IMPORTANT:** the `Session.commit()` command takes care of inserting all objects into the database **in the right order for generating all necessary foreign keys**. In this example, the newly generated primary key of the `user_account` row is applied to the `address.user_id` column appropriately:

In [57]:
session.commit()

2021-08-17 16:06:36,405 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-17 16:06:36,409 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2021-08-17 16:06:36,411 INFO sqlalchemy.engine.Engine [cached since 0.7724s ago] ('Larry', 'Larry the Lobster')
2021-08-17 16:06:36,417 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2021-08-17 16:06:36,420 INFO sqlalchemy.engine.Engine [generated in 0.00292s] ('larry.lobster@sea.com', 4)
2021-08-17 16:06:36,423 INFO sqlalchemy.engine.Engine INSERT INTO address (email_address, user_id) VALUES (?, ?)
2021-08-17 16:06:36,426 INFO sqlalchemy.engine.Engine [cached since 0.009043s ago] ('larrylob@aol.com', 4)
2021-08-17 16:06:36,430 INFO sqlalchemy.engine.Engine COMMIT


**Loading relationships**

When we called `Session.commit()`, we emitted a COMMIT for the transaction, and then per `Session.commit.expire_on_commit` **expired all objects** so that they refresh for the next transaction.

When we next access an attribute on these objects, we’ll see the SELECT emitted for the primary attributes of the row, such as when we view the newly generated primary key for the `u1` object:

In [58]:
u1.id

2021-08-17 16:06:36,444 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-17 16:06:36,453 INFO sqlalchemy.engine.Engine SELECT user_account.id AS user_account_id, user_account.name AS user_account_name, user_account.fullname AS user_account_fullname 
FROM user_account 
WHERE user_account.id = ?
2021-08-17 16:06:36,457 INFO sqlalchemy.engine.Engine [generated in 0.00378s] (4,)


4

The `u1` User object now has a **persistent** collection `User.addresses` that we may also access. 

In [59]:
u1.addresses

2021-08-17 16:06:36,473 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2021-08-17 16:06:36,477 INFO sqlalchemy.engine.Engine [cached since 0.431s ago] (4,)


[Address(id=1, email_address='larry.lobster@sea.com'),
 Address(id=2, email_address='larrylob@aol.com')]

### ORM: Using relationships in queries
---
In this section, we introduce the behavior of `relationship()` as it applies to **class level behavior of a mapped class**, where it serves in several ways to help automate the construction of SQL queries.

**Using relationships to join**

The [ex2_sqlalchemy_Core.ipynb](ex2_sqlalchemy_Core.ipynb) code introduced the usage of the `Select.join()` and `Select.join_from()` methods to compose SQL JOIN clauses. In order to describe how to join between tables, these methods either infer the ON clause based on the presence of a single unambiguous `ForeignKeyConstraint` object within the table metadata structure that links the two tables, or otherwise we may provide an explicit SQL Expression construct that indicates a specific ON clause.

When using ORM entities, an additional mechanism is available to help us set up the ON clause of a join, which is to make use of the `relationship()` objects that we set up in our `User` and `Address` mapping. The class-bound attribute corresponding to the `relationship()` may be passed as the single argument to `Select.join()`, where it serves to indicate both the right side of the join as well as the ON clause at once:

In [60]:
# SELECT statement using an ON clause to join tables (User and Address objects)
print(
    select(Address.email_address).
    select_from(User).
    join(User.addresses) # using the relationship attribute from User to fetch corresponding addresses
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


**IMPORTANT:** the presence of an ORM `relationship()` on a mapping is not used by `Select.join()` or `Select.join_from()`. If we don’t specify it, it is not used for ON clause inference. This means, if we join from `User` to `Address` without an ON clause, it works because of the `ForeignKeyConstraint` between the two mapped Table objects, not because of the `relationship()` objects on the `User` and `Address` classes:

In [61]:
# SELECT statement NOT using an ON clause to join tables (User and Address objects)
print(
    select(Address.email_address).
    join_from(User, Address) # using the ForeignKeyConstraint to perform the join
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


### ORM: Entity Aliases

If we want to associate an alias to an object, we can use the ORM `aliased()` function, which may be applied to an entity such as `User` and `Address`. This produces an **alias object internally** that’s against the original mapped Table object, while maintaining ORM functionality. The SELECT below selects from the `User` entity all objects that include two particular email addresses:

In [62]:
# importing necessary libraries
from sqlalchemy.orm import aliased

# creating aliases to the Address objects 
address_alias_1 = aliased(Address)
address_alias_2 = aliased(Address)

# SELECT statement making use of aliased tables (objects)
print(
    select(User).
    join_from(User, address_alias_1).
    where(address_alias_1.email_address == 'larrylob@aol.com').
    join_from(User, address_alias_2).
    where(address_alias_2.email_address == 'larry.lobster@sea.com')
)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account JOIN address AS address_1 ON user_account.id = address_1.user_id JOIN address AS address_2 ON user_account.id = address_2.user_id 
WHERE address_1.email_address = :email_address_1 AND address_2.email_address = :email_address_2


To make use of a `relationship()` to construct a JOIN from an aliased entity, the attribute is available from the `aliased()` construct directly.

In [63]:
# aliased User object
user_alias_1 = aliased(User)

# SELECT statement making use of the aliased object referring to User
print(
    select(user_alias_1.name).
    join(user_alias_1.addresses)
)

SELECT user_account_1.name 
FROM user_account AS user_account_1 JOIN address ON user_account_1.id = address.user_id


**Augmenting the ON criteria**

The ON clause generated by the `relationship()` construct may also be augmented with additional criteria. For example, we can use the `PropComparator.and_()` method that will be joined to the ON clause of the JOIN via AND.

Check the [documentation](https://docs.sqlalchemy.org/en/14/tutorial/orm_related_objects.html) for additional functions you can use, especially for the `PropComparator` class and **common relationship operators**.

For example, if we wanted to JOIN from `User` to `Address` but also limit the ON criteria to only certain email addresses:

In [64]:
# SELECT statement making use of AND to refine the JOIN clause
stmt = (
    select(User.fullname).
    join(User.addresses.and_(Address.email_address == 'larrylob@aol.com'))
)
# executing the command and retrieving the resulting rows
session.execute(stmt).all()

2021-08-17 16:06:36,573 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account JOIN address ON user_account.id = address.user_id AND address.email_address = ?
2021-08-17 16:06:36,576 INFO sqlalchemy.engine.Engine [generated in 0.00278s] ('larrylob@aol.com',)


[('Larry the Lobster',)]

### ORM: Entity subqueries and CTEs
---
`[from documentation]`

In the ORM, the `aliased()` construct may be used to associate an ORM entity, such as our `User` or `Address` class, with any `FromClause` concept that represents a source of rows. The preceding section illustrates using `aliased()` to associate the mapped class with an Alias of its mapped Table. Here we illustrate `aliased()` doing the same thing against both a Subquery as well as a CTE generated against a Select construct, that ultimately derives from that same mapped Table.

Below is an example of applying `aliased()` to the Subquery construct, so that ORM entities can be extracted from its rows. The result shows a series of `User` and `Address` objects, where the data for each `Address` object ultimately came from a subquery against the `address` table rather than that table directly:

In [65]:
# creating a subquery to retrieve all email addresses that match the selection criterua
subq = select(Address).where(~Address.email_address.like('%@aol.com')).subquery()
# creating an aliased object to refer to the matching email addresses
address_subq = aliased(Address, subq)
# SELECT statement using the subquery over a joined User+Address object
stmt = select(User, address_subq).join_from(User, address_subq).order_by(User.id, address_subq.id)

# opening a session to execute the SELECT statement
with Session(engine) as session:
  # executing the command and iterating over the resulting rows
  for user, address in session.execute(stmt):
    print(f"{user} => {address}")

2021-08-17 16:06:36,601 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-17 16:06:36,610 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN (SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE ?) AS anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2021-08-17 16:06:36,614 INFO sqlalchemy.engine.Engine [generated in 0.00384s] ('%@aol.com',)
User(id=4, name='Larry', fullname='Larry the Lobster') => Address(id=1, email_address='larry.lobster@sea.com')
2021-08-17 16:06:36,618 INFO sqlalchemy.engine.Engine ROLLBACK


Another example follows, which is exactly the same except it makes use of the **CTE construct** instead:


In [66]:
# creating a CTE (common table expression) to retrieve all email addresses that match the selection criteria
cte = select(Address).where(~Address.email_address.like('%@aol.com')).cte()
# creating an aliased object to refer to the matching email addresses
address_cte = aliased(Address, cte)
# SELECT statement using the subquery over a joined User+Address object
stmt = select(User, address_cte).join_from(User, address_cte).order_by(User.id, address_cte.id)

# opening a session to execute the SELECT statement
with Session(engine) as session:
  # executing the command and iterating over the resulting rows
  for user, address in session.execute(stmt):
    print(f"{user} => {address}")

2021-08-17 16:06:36,640 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-17 16:06:36,648 INFO sqlalchemy.engine.Engine WITH anon_1 AS 
(SELECT address.id AS id, address.email_address AS email_address, address.user_id AS user_id 
FROM address 
WHERE address.email_address NOT LIKE ?)
 SELECT user_account.id, user_account.name, user_account.fullname, anon_1.id AS id_1, anon_1.email_address, anon_1.user_id 
FROM user_account JOIN anon_1 ON user_account.id = anon_1.user_id ORDER BY user_account.id, anon_1.id
2021-08-17 16:06:36,651 INFO sqlalchemy.engine.Engine [generated in 0.00281s] ('%@aol.com',)
User(id=4, name='Larry', fullname='Larry the Lobster') => Address(id=1, email_address='larry.lobster@sea.com')
2021-08-17 16:06:36,656 INFO sqlalchemy.engine.Engine ROLLBACK
